In [25]:
%reset

In [26]:
from sklearn.datasets import fetch_openml
import numpy as np
import time
import torch
from scipy.optimize import linprog
from qpsolvers import solve_qp
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator
from torch.autograd import Function
import torch.nn as nn
from sklearn.model_selection import train_test_split
import sys
import StochasticGhost1
import importlib

In [27]:
xd, yd = fetch_openml('mnist_784', version=1, return_X_y=True)
xd = (xd/255).astype('float32')

/home/harsh/anaconda3/envs/ghost/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [28]:
class CustomNetwork(nn.Module):

    # For now the input data is passed as init parameters
    def __init__(self, layer_sizes, itrain, otrain, ival, oval):
        super(CustomNetwork, self).__init__()

        # Create a list of linear layers based on layer_sizes
        self.itrain = itrain
        self.otrain = otrain
        self.ival = ival
        self.oval = oval
        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))

    def forward(self, x):
        #self.input = x
        for layer in self.layers[:-1]:
            x = torch.sigmoid(layer(x))
        x = self.layers[-1](x)
        return x
    
    def compute_loss(self, Y, Y_hat):
        """
        compute loss function
        """
        # L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
        L_sum = 0.5*torch.sum(torch.square(Y - Y_hat))

        m = Y.shape[0]
        L = (1./m) * L_sum

        return L
    
    def obj_fun(self, params, minibatch):
        model_parameters = list(self.parameters())
        # with torch.no_grad():
        x = self.itrain
        y = self.otrain
        samples = np.random.choice(len(y), minibatch, replace=False)
        for i in range(len(params)):
            model_parameters[i].data = torch.Tensor(params[i]) 
        #self.compute_loss(fval, y[minibatch])
        fval = self.compute_loss(self.forward(x[samples, :]), y[samples])
        return fval.item()
    
    def obj_grad(self, params, minibatch):
        #f_grad = {}
        fgrad = []
        x = self.itrain
        y = self.otrain
        samples = np.random.choice(len(y), minibatch, replace=False)
        #with torch.no_grad():
        for name, param in self.named_parameters():
            #print(name)
            fgrad.append(torch.autograd.grad(
                self.compute_loss(self.forward(x[samples, :]), y[samples]), param, retain_graph=True)[0].view(-1))
        fgrad = torch.cat(fgrad, dim=0)
        return fgrad
    
    def conf(self, params, minibatch):
        conf_val = None
        x_val = self.ival
        y_val = self.oval
        samples = np.random.choice(len(y_val), minibatch, replace=False)
        #print(y_val.shape)
        #print(x_val.shape)
        conf_val = self.forward(x_val[minibatch, :])
        conf_val = self.compute_loss(
            self.forward(x_val[samples, :]), y_val[samples])
        """
        TODO: Compute
        
        """
        return conf_val.item()
    
    def conJ(self, params, minibatch):
        x_val = self.ival
        y_val = self.oval
        cgrad = []
        samples = np.random.choice(len(y_val), minibatch, replace=False)
        #with torch.no_grad():
        for name, param in self.named_parameters():
            #print(name)
            cgrad.append(torch.autograd.grad(
                self.compute_loss(self.forward(x_val[samples, :]), y_val[samples]), param, retain_graph=True)[0].view(-1))
        
        cgrad = torch.cat(cgrad, dim=0)
        """"
        TODO: Compute

        """
        print("I am the grad tensor and I am this long: ", len(cgrad))
        return cgrad
    


    # def solvesubp(fgrad, cval, cgrad, kap, beta, tau, n):
    #     P = tau*np.eye(n)

    #     return solve_qp(P, fgrad.numpy().reshape((n,)), cgrad.numpy().reshape((1, n)), np.array([(kap-cval.numpy())]), np.zeros((0, n)), np.zeros((0,)), -beta*np.ones((n,)), beta*np.ones((n,)))

    # def computekappa(self, cval, cgrad, rho, lamb, N):
    #     obj = np.concatenate(([1.], np.zeros((N,))))
    #     Aubt = np.concatenate((([-1.]), cgrad.numpy()))
    #     Aubt = Aubt.reshape(1, N+1)
    #     res = linprog(c=obj, A_ub=Aubt,
    #                   b_ub=[-cval.numpy()], bounds=(-rho, rho))

    #     return ((1-lamb)*max(0, cval.numpy())+lamb*max(0, res.fun))

    # def backward(self, loss, constraint, param_dict):

    #     f_grad = {}
    #     fgrad = []
    #     cgrad = []
    #     c_grad = {}
    #     param_tot = sum(p.numel() for p in self.parameters())

    #     for name, param in self.named_parameters():
    #         print(name)
    #         f_grad[name] = torch.autograd.grad(
    #             loss, param, retain_graph=True)[0]
    #         fgrad.append(torch.cat(fgrad, f_grad[name].view(-1)))

    #     # Custom backward pass to add regularization term
    #     for name, param in self.named_parameters():
    #         print(name)
    #         # Assuming param is a weight matrix
    #         c_grad[name] = torch.autograd.grad(
    #             constraint, param, retain_graph=True)[0]
    #         cgrad.append(torch.cat(cgrad, f_grad[name].view(-1)))

    #     kap = self.computekappa(
    #         constraint, cgrad, param_dict["rho"], param_dict["lamb"], param_tot)
    #     dsol = self.solvesubp(fgrad, constraint, cgrad, kap,
    #                           param_dict["rho"], param_dict["lamb"], param_tot)
    #     return dsol

    # def update(self, grad, gamma):
    #     start = 0
    #     with torch.no_grad():
    #         for name, param in self.named_parameters():
    #             print(name)
    #             end = start + param.numel()
    #             param += gamma*grad[start:end].view_as(param)
    #             start = end

    # def compute_loss(self, Y, Y_hat):
    #     """
    #     compute loss function
    #     """
    #     # L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
    #     L_sum = 0.5*np.sum(np.square(Y - Y_hat))

    #     m = Y.shape[0]
    #     L = (1./m) * L_sum

    #     return L
    

    # def getBatches(Nbatch, NSamp):
    #     mbatch1 = np.random.choice(Nbatch, 1, replace=False)
    #     mbatch2 = np.random.choice(Nbatch, 2**NSamp, replace=False)
    #     mbatch3 = np.random.choice(Nbatch, 2**NSamp, replace=False)
    #     mbatch4 = np.random.choice(Nbatch, 2**(NSamp+1), replace=False)
    #     # trainT = data_utils.TensorDataset(torch.Tensor(trainT_X.values), torch.Tensor(trainT_Y.values))
    #     # train_dataloaderT = data_utils.DataLoader(trainT, batch_size=18, shuffle=True)
    #     return [mbatch1, mbatch2, mbatch3, mbatch4]

    # def train_and_update(self, xd, yd):
    #     X_train, X_val, Y_train, Y_val = train_test_split(
    #         xd, yd, test_size=0.3, random_state=42)
    #     maxiter = 500
    #     geomp = 0.7
    #     # Example: Input size, hidden sizes, output size
    #     param_dict = {"rho": 0.8, "lamb": 0.5, "beta": 10.,
    #                   "tau": 1., "geomp": 0.7, "maxiter": 500}

    #     geomp = param_dict["geomp"]
    #     Nsamp = np.random.geometric(p=geomp)
    #     cons_bound = 10.
    #     while (2**(Nsamp+1)) > min(len(X_train), len(X_val)):
    #         Nsamp = np.random.geometric(p=geomp)

    #     mbatcht = self.getBatches(self, len(X_train), Nsamp)
    #     mbatchv = self.getBatches(self, len(X_val), Nsamp)

    #     # size = len(dataloader.dataset)
    #     # model.train()
    #     accum_loss = 0
    #     dsol = []
    #     for i in len(mbatcht):
    #         output_t = self.forward(X_train[:, mbatcht[i]])
    #         output_v = self.forward(X_val[:, mbatchv[i]])
    #         t_loss = self.compute_loss(Y_train[mbatcht[i]], output_t)
    #         constraint = output_v-cons_bound
    #         dsol.append(self.backward(t_loss, constraint, param_dict))

    #     d = dsol[0] + (dsol[3]-0.5*dsol[1]-0.5*dsol[2]) / \
    #         (geomp*((1-geomp)**Nsamp))
    #     self.update(d, geomp)

In [29]:
yd = yd.astype(int)
y = np.zeros((yd.shape[0],))
y[np.where(yd == 1)] = 1

In [30]:
def paramvals(maxiter, beta, rho, lamb, hess, tau, mbsz, numcon, geomp, stepdecay, gammazero, zeta, N, n):
    params = {
        'maxiter': maxiter,  # number of iterations performed
        'beta': beta,  # trust region size
        'rho': rho,  # trust region for feasibility subproblem
        'lamb': lamb,  # weight on the subfeasibility relaxation
        'hess': hess,  # method of computing the Hessian of the QP, options include 'diag' 'lbfgs' 'fisher' 'adamdiag' 'adagraddiag'
        'tau': tau,  # parameter for the hessian
        'mbsz': mbsz,  # the standard minibatch size, used for evaluating the progress of the objective and constraint
        'numcon': numcon,  # number of constraint functions
        'geomp': geomp,  # parameter for the geometric random variable defining the number of subproblem samples
        'stepdecay': stepdecay, # strategy for step decrease, options include 'dimin' 'stepwise' 'slowdimin' 'constant'
        'gammazero': gammazero,  # initial stepsize
        'zeta': zeta,  # parameter associated with the stepsize iteration
        'N' : N, #Train/val sample size
        'n' : n  # Total number of parameters
    }
    return params

In [31]:
import types

In [32]:
def reload_package(root_module):
    package_name = root_module.__name__

    # get a reference to each loaded module
    loaded_package_modules = dict([
        (key, value) for key, value in sys.modules.items() 
        if key.startswith(package_name) and isinstance(value, types.ModuleType)])

    # delete references to these loaded modules from sys.modules
    for key in loaded_package_modules:
        del sys.modules[key]

    # load each of the modules again; 
    # make old modules share state with new modules
    for key in loaded_package_modules:
        print("Loading ", key)
        newmodule = __import__(key)
        oldmodule = loaded_package_modules[key]
        oldmodule.__dict__.clear()
        oldmodule.__dict__.update(newmodule.__dict__)

In [38]:
import StochasticGhost1
reload_package(StochasticGhost1)

Loading  StochasticGhost1


In [39]:
maxiter = 1
layer_sizes = [70, 15, 15, 1]
#print(type(xd))
X_train, X_val, Y_train, Y_val = train_test_split(
    torch.tensor(xd.values), torch.tensor(y), test_size=0.3, random_state=42)
#print(type(X_train))
train_loss_list = []
val_loss_list = []
cons_bound = 10.
#X_train = torch.tensor()
net = CustomNetwork(layer_sizes, X_train[:, :70], Y_train, X_val[:, :70], Y_val)
nn_parameters = list(net.parameters())
initw = [param.data for param in nn_parameters]
num_param = sum(p.numel() for p in net.parameters())
print("Fuck!!!!", num_param," parameters, Go easy on me, I am just cute little pc.")
#net.train_and_update(xd, yd)
#output_t = net.forward(X_train[:, :70])
#output_v = net.forward(X_val[:, :70])
#obj = net.compute_loss(Y_train[:70], output_t)
#constraint = output_v-cons_bound
params = paramvals(maxiter=2, beta=10, rho=0.8, lamb=0.5, hess='diag', tau=1., mbsz=1,
                   numcon=1, geomp=0.7, stepdecay='dimin', gammazero=0.1, zeta=0.1, N=X_val.shape[0], n=num_param)
#print(params["N"])
#print((initw))
w, iterfs, itercs = StochasticGhost1.StochasticGhost(net.obj_fun, net.obj_grad, [net.conf], [net.conJ], initw, params)
#train_loss_list.append(train_loss)
#y_pred = net.forward(X_val)
#val_loss = net.compute_loss(Y_val, y_pred)
#val_loss_list.append(val_loss)

# net = CustomNetwork(input_size, hidden_size, output_size)

Fuck!!!! 1321  parameters, Go easy on me, I am just cute little pc.
Type of initw: <class 'list'>
LOOKIE LOOKIE:::::::::  0.1528610946724247
<class 'numpy.ndarray'>
I am the grad tensor and I am this long:  1321
<class 'numpy.ndarray'>
The cgrad shape is: (1321,)
The Aubt shape is: (1322,)
<class 'numpy.float64'> <class 'numpy.ndarray'> <class 'numpy.float64'> <class 'numpy.ndarray'>
[-0.         -0.         -0.         ... -0.2797214  -0.24064624
 -0.55292153]
[0.         0.         0.         ... 0.22617571 0.19458051 0.4470785 ]
<class 'numpy.ndarray'>
I am the grad tensor and I am this long:  1321
<class 'numpy.ndarray'>
The cgrad shape is: (1321,)
The Aubt shape is: (1322,)
<class 'numpy.float64'> <class 'numpy.ndarray'> <class 'numpy.float64'> <class 'numpy.ndarray'>
[0.         0.         0.         ... 0.90470284 0.77832204 1.788314  ]
[0.         0.         0.         ... 0.90470284 0.77832204 1.78831398]
<class 'numpy.ndarray'>
I am the grad tensor and I am this long:  1321
<

In [63]:
print(iterfs)
print(itercs)

[0.10296745]
[0.10296745]


In [24]:
yd = yd.astype(int)
print(torch.tensor(yd.values))

tensor([5, 0, 4,  ..., 4, 5, 6])
